In [ ]:
import numpy as np

# Given data
costs = np.array([
    [92.63, 104.03, 145.95, 112.43, 107.80, 112.19],
    [160.20, 93.25, 148.88, 113.61, 103.45, 111.85],
    [186.70, 122.31, 112.31, 135.98, 127.76, 133.35],
    [127.34, 84.84, 122.51, 73.34, 87.84, 91.04],
    [152.64, 95.15, 144.73, 107.62, 89.15, 107.00],
    [252.78, 162.24, 236.36, 177.62, 168.96, 149.24]
])

capacity = np.array([22.0, 3.7, 4.5, 47.0, 18.5, 5.0])
demand = np.array([3.00, 2.60, 16.00, 20.00, 26.40, 11.90])

def vam(costs, capacity, demand):
    allocation = np.zeros_like(costs)
    costs_copy = costs.copy()

    while np.sum(capacity) > 0 and np.sum(demand) > 0:
        row_diff = np.apply_along_axis(lambda x: x[x != np.inf].min() if len(x[x != np.inf]) > 1 else np.inf, 1, costs_copy)
        col_diff = np.apply_along_axis(lambda x: x[x != np.inf].min() if len(x[x != np.inf]) > 1 else np.inf, 0, costs_copy)

        if row_diff.min() <= col_diff.min():
            i = np.argmin(row_diff)
            j = np.argmin(costs_copy[i])
        else:
            j = np.argmin(col_diff)
            i = np.argmin(costs_copy[:, j])

        min_transfer = min(capacity[i], demand[j])
        allocation[i, j] = min_transfer
        if capacity[i] == min_transfer:
            costs_copy[i, :] = np.inf
        if demand[j] == min_transfer:
            costs_copy[:, j] = np.inf
        capacity[i] -= min_transfer
        demand[j] -= min_transfer

    return allocation

# Calculate framework matrix for 100% capacity
result_100 = vam(costs, capacity.copy(), demand.copy())
print("Framework Matrix for 100% capacity:")
print(result_100)
cost_result_100 = result_100 * costs
print(f"\nThe projected cost for 100% capacity: \n{cost_result_100}")
print(f"The projected cost for 100% capacity: {round(np.sum(cost_result_100)/100,3)} million dollars")


capacity_85 = capacity * 0.85
result_85 = vam(costs, capacity_85.copy(), demand.copy())
print("\nFramework Matrix for 85% capacity:")
print(result_85)
cost_result_85 = result_85 * costs
print(f"\nThe projected cost for 85% capacity: \n{cost_result_85}")
print(f"The projected cost for 85% capacity: {round(np.sum(cost_result_85)/100,3)} million dollars")


# using 90% capacity, but use chile
capacity_90 = [s * 0.9 for s in capacity]
# Use the VAM method
result_with_chile_90 = vam(costs, capacity_90, demand)
# Print the Framework Matrix
print("Framework Matrix with Chile and at 90% capacity:")
print(result_with_chile_90)
# get cost for production plan with Chile at 90% capacity
cost_Chile_90 = result_with_chile_90 * costs / 100
print(f"\nThe projected cost for 90% capacity with Chile: \n{cost_Chile_90}")
print(f"The projected cost for 90% capacity with Chile: {round(np.sum(cost_Chile_90),3)} million dollars")


Framework Matrix for 100% capacity:
[[ 3.   0.   6.9  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   4.5  0.   0.   0. ]
 [ 0.   2.6  0.  20.  24.4  0. ]
 [ 0.   0.   4.6  0.   2.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]]

The projected cost for 100% capacity: 
[[ 277.89     0.    1007.055    0.       0.       0.   ]
 [   0.       0.       0.       0.       0.       0.   ]
 [   0.       0.     505.395    0.       0.       0.   ]
 [   0.     220.584    0.    1466.8   2143.296    0.   ]
 [   0.       0.     665.758    0.     178.3   1273.3  ]
 [   0.       0.       0.       0.       0.       0.   ]]
The projected cost for 100% capacity: 77.384 million dollars

Framework Matrix for 85% capacity:
[[ 3.     0.    12.175  0.     0.     2.08 ]
 [ 0.     0.     0.     0.     0.     3.145]
 [ 0.     0.     3.825  0.     0.     0.   ]
 [ 0.     2.6    0.    20.    17.35   0.   ]
 [ 0.     0.     0.     0.     9.05   6.675]
 [ 0.     0.     0.     0.     0.     0.   ]]

The projected

In [ ]:
import numpy as np

# Given data
costs = np.array([
    [92.63, 104.03, 145.95, 112.43, 107.80, 112.19],
    [160.20, 93.25, 148.88, 113.61, 103.45, 111.85],
    [186.70, 122.31, 112.31, 135.98, 127.76, 133.35],
    [127.34, 84.84, 122.51, 73.34, 87.84, 91.04],
    [152.64, 95.15, 144.73, 107.62, 89.15, 107.00],
    [252.78, 162.24, 236.36, 177.62, 168.96, 149.24]
])

capacity = np.array([22.0, 3.7, 4.5, 47.0, 18.5, 5.0])
demand = np.array([3.00, 2.60, 16.00, 20.00, 26.40, 11.90])



def vam(costs, capacity, demand):
    allocation = np.zeros((len(capacity), len(demand)))
    costs_copy = np.array(costs)
    last_allocation = -1  # To track the change in allocation after each iteration

    while sum(capacity) > 0 and sum(demand) > 0:
        #If no change in allocation from the last iteration, break the loop
        if np.sum(allocation) == last_allocation:
            break

        last_allocation = np.sum(allocation)

        row_diff = []
        col_diff = []

        # Compute the constraints for rows
        for row in costs_copy:
            sorted_row = sorted([x for x in row if x != float('inf')])
            if len(sorted_row) >= 2:
                row_diff.append(sorted_row[1] - sorted_row[0])
            else:
                row_diff.append(0)

        # Compute the constraints for columns
        for i in range(len(demand)):
            sorted_col = sorted([x for x in costs_copy[:, i] if x != float('inf')])
            if len(sorted_col) >= 2:
                col_diff.append(sorted_col[1] - sorted_col[0])
            else:
                col_diff.append(0)

        # Find the maximum constraints
        if max(row_diff) > max(col_diff):
          idx = row_diff.index(max(row_diff))
          j = np.argmin(costs_copy[idx, :])
        else:
          max_col_diff = max(col_diff)
          j = col_diff.index(max_col_diff)
        # To handle ties, find the first column with the maximum constraint
        for i in range(len(col_diff)):
          if col_diff[i] == max_col_diff:
            j = i
            break
        idx = np.argmin(costs_copy[:, j])

        # Assign the minimum of demand and capacity to the cell
        alloc = min(capacity[idx], demand[j])
        allocation[idx][j] = alloc
        capacity[idx] -= alloc
        demand[j] -= alloc

        # Set the costs of exhausted row or column to a high value
        if capacity[idx] == 0:
            costs_copy[idx, :] = float('inf')
        if demand[j] == 0:
            costs_copy[:, j] = float('inf')

    return allocation

# Calculate framework matrix for 100% capacity
result_100 = vam(costs, capacity.copy(), demand.copy())
print("Framework Matrix for 100% capacity:")
print(result_100)
cost_result_100 = result_100 * costs
print(f"\nThe projected cost for 100% capacity: \n{cost_result_100}")
print(f"The projected cost for 100% capacity: {round(np.sum(cost_result_100)/100,3)} million dollars")


capacity_85 = capacity * 0.85
result_85 = vam(costs, capacity_85.copy(), demand.copy())
print("\nFramework Matrix for 85% capacity:")
print(result_85)
cost_result_85 = result_85 * costs
print(f"\nThe projected cost for 85% capacity: \n{cost_result_85}")
print(f"The projected cost for 85% capacity: {round(np.sum(cost_result_85)/100,3)} million dollars")


# using 90% capacity, but use chile
capacity_90 = [s * 0.9 for s in capacity]
# Use the VAM method
result_with_chile_90 = vam(costs, capacity_90, demand)
# Print the Framework Matrix
print("Framework Matrix with Chile and at 90% capacity:")
print(result_with_chile_90)
# get cost for production plan with Chile at 90% capacity
cost_Chile_90 = result_with_chile_90 * costs / 100
print(f"\nThe projected cost for 90% capacity with Chile: \n{cost_Chile_90}")
print(f"The projected cost for 90% capacity with Chile: {round(np.sum(cost_Chile_90),3)} million dollars")





Framework Matrix for 100% capacity:
[[ 3.   0.   0.   0.   3.2  0. ]
 [ 0.   0.   0.   0.   3.7  0. ]
 [ 0.   0.   4.5  0.   0.   0. ]
 [ 0.   2.6 11.5 20.   1.  11.9]
 [ 0.   0.   0.   0.  18.5  0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

The projected cost for 100% capacity: 
[[ 277.89     0.       0.       0.     344.96     0.   ]
 [   0.       0.       0.       0.     382.765    0.   ]
 [   0.       0.     505.395    0.       0.       0.   ]
 [   0.     220.584 1408.865 1466.8     87.84  1083.376]
 [   0.       0.       0.       0.    1649.275    0.   ]
 [   0.       0.       0.       0.       0.       0.   ]]
The projected cost for 100% capacity: 74.278 million dollars

Framework Matrix for 85% capacity:
[[ 3.     0.     4.125  0.    10.13   0.   ]
 [ 0.     2.6    0.     0.     0.545  0.   ]
 [ 0.     0.     3.825  0.     0.     0.   ]
 [ 0.     0.     8.05  20.     0.    11.9  ]
 [ 0.     0.     0.     0.    15.725  0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]]

The projected

In [ ]:
import numpy as np

# Given data
costs = np.array([
    [92.63, 104.03, 145.95, 112.43, 107.80, 112.19],
    [160.20, 93.25, 148.88, 113.61, 103.45, 111.85],
    [186.70, 122.31, 112.31, 135.98, 127.76, 133.35],
    [127.34, 84.84, 122.51, 73.34, 87.84, 91.04],
    [152.64, 95.15, 144.73, 107.62, 89.15, 107.00],
    [252.78, 162.24, 236.36, 177.62, 168.96, 149.24]
])

capacity = np.array([22.0, 3.7, 4.5, 47.0, 18.5, 5.0])
demand = np.array([3.00, 2.60, 16.00, 20.00, 26.40, 11.90])



def vam(costs, capacity, demand):
    allocation = np.zeros((len(capacity), len(demand)))
    costs_copy = np.array(costs)
    last_allocation = -1  # To track the change in allocation after each iteration

    while sum(capacity) > 0 and sum(demand) > 0:
        #If no change in allocation from the last iteration, break the loop
        if np.sum(allocation) == last_allocation:
            break

        last_allocation = np.sum(allocation)

        row_diff = []
        col_diff = []

        # Compute the constraints for rows
        for row in costs_copy:
            sorted_row = sorted([x for x in row if x != float('inf')])
            if len(sorted_row) >= 2:
                row_diff.append(sorted_row[1] - sorted_row[0])
            else:
                row_diff.append(0)

        # Compute the constraints for columns
        for i in range(len(demand)):
            sorted_col = sorted([x for x in costs_copy[:, i] if x != float('inf')])
            if len(sorted_col) >= 2:
                col_diff.append(sorted_col[1] - sorted_col[0])
            else:
                col_diff.append(0)

        # Find the maximum constraints
        if max(row_diff) > max(col_diff):
          idx = row_diff.index(max(row_diff))
          j = np.argmin(costs_copy[idx, :])
        else:
          max_col_diff = max(col_diff)
          j = col_diff.index(max_col_diff)
        # To handle ties, find the first column with the maximum constraint
        for i in range(len(col_diff)):
          if col_diff[i] == max_col_diff:
            j = i
            break
        idx = np.argmin(costs_copy[:, j])

        # Assign the minimum of demand and capacity to the cell
        alloc = min(capacity[idx], demand[j])
        allocation[idx][j] = alloc
        capacity[idx] -= alloc
        demand[j] -= alloc

        # Set the costs of exhausted row or column to a high value
        if capacity[idx] == 0:
            costs_copy[idx, :] = float('inf')
        if demand[j] == 0:
            costs_copy[:, j] = float('inf')

    return allocation


# set capacity for Chile to be 0
reduced_capacity = [22.0, 3.7, 0, 47.0, 18.5, 5.0]
reduced_capacity_90 = [s * 0.9 for s in reduced_capacity]

# Use the VAM method
result_without_chile = vam(costs, reduced_capacity_90, demand)
# Print the Framework Matrix
print("Framework Matrix without Chile and at 90% capacity:")
print(result_without_chile)
# get cost for production plan without Chile at 90% capacity
cost_noChile_90 = result_without_chile * costs / 100
print(f"\nThe projected cost for 90% capacity without Chile: \n{cost_noChile_90}")
print(f"The projected cost for 90% capacity without Chile: {round(np.sum(cost_noChile_90),3)} million dollars")

Framework Matrix without Chile and at 90% capacity:
[[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]

The projected cost for 90% capacity without Chile: 
[[ 2.7789   0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.      14.668    0.      10.83376]
 [ 0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.     ]]
The projected cost for 90% capacity without Chile: 28.281 million dollars
